In [ ]:
print("ADITYA RAM S H\n1BM22CS019\n")
class Variable:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return isinstance(other, Variable) and self.name == other.name

    def __hash__(self):
        return hash(self.name)

    def __repr__(self):
        return self.name


class Term:
    def __init__(self, functor, *args):
        self.functor = functor
        self.args = args

    def __eq__(self, other):
        return isinstance(other, Term) and self.functor == other.functor and self.args == other.args

    def __repr__(self):
        return f"{self.functor}({', '.join(map(str, self.args))})"


def apply_substitution(term, subst):
    """
    Apply the current substitution to a term.
    """
    if isinstance(term, Variable):
        return subst.get(term, term)
    elif isinstance(term, Term):
        return Term(term.functor, *[apply_substitution(arg, subst) for arg in term.args])
    return term


def occurs_check(variable, term, subst):
    """
    Checks if a variable appears in a term, including terms in the substitution.
    """
    if variable == term:
        return True
    if isinstance(term, Variable):
        return term in subst and occurs_check(variable, subst[term], subst)
    if isinstance(term, Term):
        return any(occurs_check(variable, arg, subst) for arg in term.args)
    return False


def unify_terms(t1, t2, subst):
    """
    Try to unify two terms with the current substitution.
    """
    t1 = apply_substitution(t1, subst)
    t2 = apply_substitution(t2, subst)

    if isinstance(t1, Variable):
        if t1 == t2:
            return subst
        if occurs_check(t1, t2, subst):
            print(f"Occurs check failed for {t1} in {t2}")
            return None
        print(f"Substituting {t1} with {t2}")
        subst[t1] = t2
        return subst

    if isinstance(t2, Variable):
        return unify_terms(t2, t1, subst)

    if isinstance(t1, Term) and isinstance(t2, Term):
        if t1.functor != t2.functor:
            print(f"Different functors: {t1.functor} != {t2.functor}")
            return None
        if len(t1.args) != len(t2.args):
            print(f"Different arities: {len(t1.args)} != {len(t2.args)}")
            return None
        for arg1, arg2 in zip(t1.args, t2.args):
            subst = unify_terms(arg1, arg2, subst)
            if subst is None:
                return None
        return subst

    print(f"Cannot unify {t1} with {t2}")
    return None


def unify(*terms):
    """
    Unify multiple terms by applying the unification process iteratively.
    """
    subst = {}
    for i in range(len(terms) - 1):
        print(f"Unifying {terms[i]} with {terms[i+1]}")
        subst = unify_terms(terms[i], terms[i + 1], subst)
        if subst is None:
            print("Unification failed!")
            return None
    print(f"Unification successful! Substitution: {subst}")
    return subst


# Example
var_x = Variable('x')
var_y = Variable('y')

psi1 = Term('f', var_x, Term('g', var_y))

psi2 = Term('f', 'a', Term('g', 'b'))

subst = unify(psi1, psi2)
if subst is None:
    print("Unification failed!")
else:
    print("Unification succeeded with substitution:", subst)


ADITYA RAM S H
1BM22CS019

Unifying f(x, g(y)) with f(a, g(b))
Substituting x with a
Substituting y with b
Unification successful! Substitution: {x: 'a', y: 'b'}
Unification succeeded with substitution: {x: 'a', y: 'b'}
